# Sudoku Solver
---

Sudoku is number puzzle with 9X9 grid with 9 3X3 subgrids. Some of the squares are filled with numbers.
The objective of the puzzle is to fill all the 81 squares with numbers from 1-9 in a way that no number gets repeated in each row, column or box. 


In [1]:
#

from SudokuSolver import SudokuSolver 

squares   = str
gridValDict = dict
cols = '123456789'
rows      = 'ABCDEFGHI'
#cols      = digits

In [2]:
def cross(A,B) -> tuple:
    return tuple(a+b for a in A for b in B)

## Representation of puzzle
---

The puzzle is represented by as mentioned in Stuart Russell and Peter Norvig's 'Artificial Intelligence: A Modern Approach' book.

    - 81 squares [A1', 'A2', ... 'I9']
    - 3X3 subgrids are represented as boxes 
    - units are  rows, columns and boxes(subgrids)  
    - peers of a square represents are the squares that are present in the same row or colum or subgrid

In [3]:
squares   = cross(rows, cols)
#list of tuples - [('A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3'),...] 
#all 3x3 boxes in puzzle - total 9
all_boxes = [cross(rs, cs)  for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')] 
#list of tuples - [('A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'),..]
#all  rows, columns and boxes - total 27
all_units = [cross(rows, c) for c in cols] + [cross(r, cols) for r in rows] + all_boxes

"""
dictionary holding units for each square - {'A1': (('A1',...,'I1'), ('A1',...,'A9'), ('A1',...,'C3')),
                                            'A2': (('A2',...,'I2'), ('A1',...,'A9'), ('A1',...,'C3')),
                                            .
                                            .
                                            'I9':(('A9',...,'I9'),  ('I1',...,'I9'), ('G7',...,'I9'))}

Total - 81 units
"""

units     = {s: tuple(u for u in all_units if s in u) for s in squares}
# Dictionary holding set of peers - {'A1': {'A2', 'A3',.....,I1},.....'I9':{A9,...,I8} }
# Total 81 units

peers     = {s: set().union(*units[s]) - {s} for s in squares}

In [4]:
# Module to display the Sudoku grid

def display(values):
    "Display these values as a 2-D grid."
    # rows      = 'ABCDEFGHI'
    # cols      = '123456789'

    width = 1+max(len(values[s]) for s in squares)
    
    
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print ( ''.join(values[r+c].center(width)+('|' if c in '36' else '')
                    for c in cols) )
        if r in 'CF': print(line)
    print()



## Constraint Satisfaction Problem
---
Constraint Satisfaction Probelm is a mathematical problem that can be solved by finding the values of all the variables with respect to the constraints. The Constraint Satisfaction problem consists of

- **Variables** - Variables are the states. In the case of Sudoku the 81 squares from A1 to I9 are variables
- **Domains**  - Domains are the set of values that can be assigned to the variables. In Sudoku, domain is the values from 1-9 which are the possible values of each state
- **Constraints** - Constraint is the set of restrictions applied on the problem. In sudoku, The numbers are constrained in a way that no number should be repeated in each Columns, Rows and subgrid(box)


Sudoku is considered as a partial assignment problem since few of the grids are already filled.

There are different types of constraints. 
* **Unary Constraint** - Constraint on a single variable
* **Binary Constraint** - Constraint is related with two variables
* **Global Constraint** - This contains arbitrary number of variables. Alldiff constraint is a global constraint which means all the variables are different. The contraint in sudoku problem is alldiff constraint since all the variables in each unit(row, column, box) should be different.




    

In [5]:
# Example sudoku solved - one easy and one hard

grid1 = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
grid2 = '..53.....8......2..7..1.5..4....53...1..7...6..32...8..6.5....9..4....3......97..'

solver = SudokuSolver(squares, all_units, peers)
constraint = 3
gridValDict = solver.gridtoValues(grid1)
consProp = solver.reduceGridVal(gridValDict, constraint)
print('-'*60)
print("\tPuzzle after contraint propagation")
print('-'*60)
display(consProp)
print('-'*60)
print("\tPuzzle after backpropagation")
print('-'*60)
solution = solver.backtrack(consProp, constraint)
display(solution)

# solution = solver.solvePuzzle(grid)
# display(solution)

------------------------------------------------------------
	Puzzle after contraint propagation
------------------------------------------------------------
4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 

------------------------------------------------------------
	Puzzle after backpropagation
------------------------------------------------------------
4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 



In the above examples, the easiest puzzle is solved by the constraint propagation itself. But the hard one is not solve with constraint propagation alone. So backtracking is done to search the solution.

###Constraint Propagation
Constraint propagation is an inference that can be used to reduce the domain values for the variable, that will inturn reduce the domain value of another variable and so on.

In the sudoku solver module, I have implemented three constraint propagation methods and backtrackking for searching the required combination.
Simple naive backtracking alone will be able to find the solution  for the sudoku puzzles. But the drawback ther ewould be the time complexity. It will search each and every combination from scratch. That would be n*9!*(81-n)! where n is the number of digits filled already filled.
Inorder to fasten the process, The constraint propagation will be helpful to reduce the domain values for some variables by appying some strategies. I applied the following strategies and compared its results.

- **Elimination Strategy**
    - If there is a value present in one of the squares (variable), then eliminate the value from all its peer's (ie squares belong to the same row/column/box) domains.


- **Single posssibility strategy**
    - In this strategy, for a square, if there is only one possible number then assign the number in that square(remove all the other values in the variables domain).
    

- **Naked twins rule**
    - This is a little complicated strategy that will be helpful for solving sone hard puzzles. When two squares in the same unit(two/column/box) has two domain values and the same domain values, then those two values can occur in only those two squares. So those two values can be removed from domain of all the other squares that are peers.


After the constaint propagation most of the easy puzzles got solved. But for the hard problem again had to do backtracking.
# Backtracking Search
Back tracking search uses a depth first search to try all values till the bottom and then backtrack if these is any inconsistency (ie no value can be assigned to the variable to satisfy the constraint)



# Performance comparison for the solver with various combinations of methods
---


In [6]:
import time

In [7]:
# Module to calculate time taken to solve all the puzzles in ths given file
def timetaken(filename, constraint):
    file1 = open(filename, 'r')
    Lines = file1.readlines()
    start = time.time()
    count = 0
    for line in Lines:
        count = count+1
        solver.solvePuzzle(line, constraint)
        
    end = time.time()
    avgTime = (end-start)/count
    return avgTime
    # print(f"The time of execution grids in  {filename} : {end-start}")
    # print(f"Average execution time grids in  filename {filename}: {(end-start)/count }")


In [8]:
# Generates table give dictionary containing filename and Average execution time

def generateTable(execTime):   
    # Print the names of the columns.
    print ("{:<15} {:<15}".format('Filename','Average execution time'))
    print ("{:<35} ".format('-'*35))
    # print each data item.
    for key, value in execTime.items():
        print ("{:<15} {:<15}".format(key,value))

In [10]:
def procTime(constraint):
    execTime = {}
    files = ['Easy50.txt', 'Hard.txt', 'Hardest.txt']
    for file in files:
        execTime[file] = timetaken(file,constraint)

    generateTable(execTime)  



In [11]:
#Processing time calculated when using all 3 constraints
procTime(3)

Filename        Average execution time
----------------------------------- 
Easy50.txt      0.006803736686706543
Hard.txt        0.7059818518789192
Hardest.txt     0.20066779310053046


In [12]:
#Processing time calculated when using 2 constraints
procTime(2)

Filename        Average execution time
----------------------------------- 
Easy50.txt      0.007482290267944336
Hard.txt        1.4315458046762566
Hardest.txt     0.1999732797796076


In [ ]:
#Processing time calculated when using 1 constraint
procTime(1)

In [12]:
solver = SudokuSolver(squares, all_units, peers)
gridValDict = solver.gridtoValues(grid1)
constraint = 2
display(solver.solvePuzzle(gridValDict, constraint))
#print(solver.backtrack_counter)

4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 



In [100]:
execTime

{'Easy50.txt': (0.36340928077697754, 0.007268185615539551),
 'Hard.txt': (68.83200240135193, 0.7245473936984413),
 'Hardest.txt': (2.440561532974243, 0.22186923027038574)}

In [58]:
timetaken('Hard.txt')

The time of execution grids in  Hard.txt : 59.24965763092041
Average execution time grids in  filename Hard.txt: 0.6236806066412675


In [71]:
timetaken('Easy50.txt')

0.3391704559326172

In [60]:
timetaken('Hardest.txt')

The time of execution grids in  Hardest.txt : 2.1409449577331543
Average execution time grids in  filename Hardest.txt: 0.19463135979392313


In [47]:
file1 = open('Easy50.txt', 'r')
Lines = file1.readlines()

count = 0
for line in Lines:
    count = count+1
    print(line)

..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..

2...8.3...6..7..84.3.5..2.9...1.54.8.........4.27.6...3.1..7.4.72..4..6...4.1...3

......9.7...42.18....7.5.261..9.4....5.....4....5.7..992.1.8....34.59...5.7......

.3..5..4...8.1.5..46.....12.7.5.2.8....6.3....4.1.9.3.25.....98..1.2.6...8..6..2.

.2.81.74.7....31...9...28.5..9.4..874..2.8..316..3.2..3.27...6...56....8.76.51.9.

1..92....524.1...........7..5...81.2.........4.27...9..6...........3.945....71..6

.43.8.25.6.............1.949....4.7....6.8....1.2....382.5.............5.34.9.71.

48...69.2..2..8..19..37..6.84..1.2....37.41....1.6..49.2..85..77..9..6..6.92...18

...9....2.5.1234...3....16.9.8.......7.....9.......2.5.91....5...7439.2.4....7...

..19....39..7..16..3...5..7.5......9..43.26..2......7.6..1...3..42..7..65....68..

...1254....84.....42.8......3.....95.6.9.2.1.51.....6......3.49.....72....1298...

.6234.75.1....56..57.....4.....948..4.......6..583.....3.....91..64....7.59.8326.

3...

**References**
1. https://en.wikipedia.org/wiki/Sudoku

2. https://norvig.com/sudoku.html